In [1]:
import numpy
import matplotlib
import math
%matplotlib inline

In [2]:
def vel(Vdot, A):
    v_fluid = Vdot/A
    return v_fluid

In [3]:
def re(v_fluid, L, nu):
    re_n = v_fluid * L/nu
    
    if re_n <= 5*10**5:
        laminar = 1
    else: 
        laminar = 0
    re_prop = [re_n, laminar]
    return re_prop 

In [4]:
def nusselt(re_n, pr_n):
    x = 1/3
    nusselt_n = 0.664* math.pow(re_n, 0.5) * math.pow(pr_n, x)
    return nusselt_n

In [5]:
def coh(nusselt_n, k, L):
    h = nusselt_n * k / L
    return h

In [6]:
def r_cond(L, k, A):
    r_conduction = L/(k*A)
    return r_conduction

In [7]:
def r_multi_fin(t, w, L, h, N, k, fin_surface):
    P = 2 * w + 2 * t
    A_c = t * w
    A_f = 2 * w * L + t * w
    A_b = (N-1) * t * w
    A_t = N * A_f + A_b
    m = ((h * P)/(k * A_c))**.5
    M = (h * P * k * A_c)**.5
    
    qf = M * ((math.sinh(m*L) + (h/(m*k)) * math.cosh(m*L)))/(math.cosh(m*L) + (h/m*k) * math.sinh(m*L))
    q_max = h * A_f
    etaF = qf/q_max
    etaO = 1 - (N*A_f)/A_t * (1 - etaF)
           
    R_fin = 1/(etaO * h * A_t)
    R_fin_inv = R_fin**-1
    R_inv_fin = fin_surface * R_fin_inv   
    
    return R_inv_fin

In [8]:
def pr(Cp, u, k):
    Pr = Cp*u/k
    return Pr

In [9]:
def h(k, L, re_n, pr_n):
    x = 1/3
    h = (k/L) * 0.664 * math.pow(re_n, 0.5) * math.pow(pr_n, x)
    return h

In [10]:
## Material and Chip Properties 
q = 130                           #Power of the Chip (W)
t_inf = 38                        #Temperature inside the Case as Specified by Intel (C)

L_paste = 50 * (10**-6)           #Length of the Paste (m)
k_paste = 8.9                     #Thermal Conductivity of Paste (W/m^2 K)
A_paste = 0.0405 * 0.03674        #Area of Paste (m^2)

k_6061 = 175                      #Thermal Conductivity of 6061 Aluminum

Cp_air = 1007                     #Specific Heat of Air J/kg*K
nu_air = 17.02 * 10**-6           #Kinimatic Viscosity of Air m^2/s
mu_air = 19.18*10**-6             #Dynamic Viscosity of Air kg/m*s 
k_air = 26.62*10**-3              #Thermal Conductivity of Air (W/m^2 K)

In [11]:
## Air Cooling Properties
L_core = 0.01728
A_core = 0.0375**2
t_fin = 0.0015
w_fin = 0.050
L_fin = 0.040
N_fin = 13
N_fin_surfaces = 4

V_dot_fan = 0.03912444 #82.9CFM
A_fan = math.pi * 0.25 * (.12**2 - 0.035**2)
u_fan = vel(Vdot = V_dot_fan, A = A_fan)

re_air = re(v_fluid = u_fan, L = w_fin, nu = nu_air)
Pr_air = pr(Cp = Cp_air, u = mu_air, k = k_air)
Nusselt_air = nusselt(re_n = re_air[0], pr_n = Pr_air)
h_air = h(k = k_air, L = w_fin, re_n = re_air[0], pr_n = Pr_air)
r_paste = r_cond(L = L_paste, k = k_paste, A = A_paste)
r_core = r_cond(L = L_core, k = k_6061, A = A_core)
r_fin = r_multi_fin(t = t_fin, w = w_fin, L = L_fin, h = h_air, N = N_fin, k = k_6061, fin_surface = N_fin_surfaces)
r_eq = r_paste + r_fin + r_core
t_dtc_chip = q * r_eq + t_inf
t_dtc_chip

67.3286557083139